In [1]:
import pandas as pd
import requests
from plyer import notification
from datetime import datetime

In [2]:
def py_notification(level, database, step):
    now =str(datetime.now())
    msg = f"We have encountered an error when loading the database {database} on the {step} step.\nTimestamp: {now}."

    if level == 1:
        title = "Level 1 alert."
    elif level == 2:
        title = "Attention! Level 2 alert."
    elif level == 3:
        title = "DANGER! Level 3 alert."
    else:
        title = "UNKNOWN LEVEL of alert! Please contact your IT support."

    notification.notify(
        title = title,
        message = msg,
        app_name = 'Alert',
        timeout = 5
    )

In [3]:
def validate_data(df, expected_columns, database):
    for column in expected_columns:
        if column not in df.columns:
            py_notification(3, database, 'VALIDAÇÃO DE DADOS')
            return False
    return True

In [4]:
# Ler CEPs do arquivo CSV
df_ceps = pd.read_csv('ceps.csv')
ceps = df_ceps['cep'].tolist()

In [5]:
data_table_1 = []
log_ceps = {'status_200' : [], 'status_404' : []}

for cep in ceps:
    url = f'https://brasilapi.com.br/api/cep/v1/{cep}'
    response = requests.get(url)
    if response.status_code == 200:
        data_table_1.append(response.json())
        log_ceps['status_200'].append(cep)
    else:
        log_ceps['status_404'].append(cep)

    if len(log_ceps['status_404']) > 0:
        py_notification(1, 'CEPs', 'EXTRAÇÃO DE DADOS')

In [6]:
df_table_1 = pd.DataFrame(
    {
    'cep': [item['cep'] for item in data_table_1],
    'estado': [item['state'] for item in data_table_1],
    'cidade': [item['city'] for item in data_table_1],
    'bairro': [item['neighborhood'] for item in data_table_1],
    'rua': [item['street'] for item in data_table_1]
    }
)

df_table_1

,cep,estado,cidade,bairro,rua
0,27511630,RJ,Resende,Barbosa Lima,Rua Engenheiro Jacinto Lameira Filho
1,27323330,RJ,Barra Mansa,Ano Bom,Rua Francisco Calderaro Filho


In [7]:
states = df_table_1['estado'].unique()

data_table_2 = []

data_table_2 = []
log_cities = {'status_200': [], 'status_404': []}

for state in states:
    url = f'https://brasilapi.com.br/api/ibge/municipios/v1/{state}?providers=dados-abertos-br,gov,wikipedia'
    response = requests.get(url)
    if response.status_code == 200:
        data_table_2.extend(response.json())
        log_cities['status_200'].append(state)
    else:
        log_cities['status_404'].append(state)

if len(log_cities['status_404']) > 0:
    py_notification(1, 'STATES', 'EXTRAÇÃO DE DADOS')

df_table_2 = pd.DataFrame(
    {
    'estado': state,
    'nome': [item['nome'] for item in data_table_2],
    'codigo_ibge': [item['codigo_ibge'] for item in data_table_2]
    }
)

df_table_2

,estado,nome,codigo_ibge
0,RJ,ANGRA DOS REIS,3300100
1,RJ,APERIBE,3300159
2,RJ,ARARUAMA,3300209
3,RJ,AREAL,3300225
4,RJ,ARMACAO DOS BUZIOS,3300233
...,...,...,...
87,RJ,TRES RIOS,3306008
88,RJ,VALENCA,3306107
89,RJ,VARRE-SAI,3306156
90,RJ,VASSOURAS,3306206


In [8]:
data_table_3 = []
log_states = {'status_200': [], 'status_404': []}

for state in states:
    url = f'https://brasilapi.com.br/api/ibge/uf/v1/{state}'
    response = requests.get(url)
    if response.status_code == 200:
        data_table_3.append(response.json())
        log_states['status_200'].append(state)
    else:
        log_states['status_404'].append(state)

if len(log_states['status_404']) > 0:
    py_notification(1, 'API - Estados', 'EXTRAÇÃO DE DADOS')

df_table_3 = pd.DataFrame({
    'id': [item['id'] for item in data_table_3],
    'sigla': [item['sigla'] for item in data_table_3],
    'nome': [item['nome'] for item in data_table_3],
    'id_regiao': [item['regiao']['id'] for item in data_table_3],
    'sigla_regiao': [item['regiao']['sigla'] for item in data_table_3],
    'nome_regiao': [item['regiao']['nome'] for item in data_table_3]
})

df_table_3

,id,sigla,nome,id_regiao,sigla_regiao,nome_regiao
0,33,RJ,Rio de Janeiro,3,SE,Sudeste


In [9]:
try:
    assert validate_data(df_table_1, ['cep', 'estado', 'cidade', 'bairro', 'rua'], 'df_table_1'), "Erro na validação da tabela df_table_1"
except AssertionError as e:
    print(e)

try:
    assert validate_data(df_table_2, ['estado', 'nome', 'codigo_ibge'], 'df_table_2'), "Erro na validação da tabela df_table_2"
except AssertionError as e:
    print(e)

try:
    assert validate_data(df_table_3, ['id', 'sigla', 'nome', 'id_regiao', 'sigla_regiao', 'nome_regiao'], 'df_table_3'), "Erro na validação da tabela df_table_3"
except AssertionError as e:
    print(e)

In [10]:
from sqlalchemy import create_engine

In [11]:
# Armazenar os dados no banco de dados
engine = create_engine('sqlite:///data.db')
df_table_1.to_sql('cep_data', engine, index=False, if_exists='replace')
df_table_2.to_sql('municipios_data', engine, index=False, if_exists='replace')
df_table_3.to_sql('estados_data', engine, index=False, if_exists='replace')

1

In [12]:
with pd.ExcelWriter('dados_extraidos_ceps.xlsx') as writer:
    df_table_1.to_excel(writer, sheet_name='CEP', index=False)
    df_table_2.to_excel(writer, sheet_name='Municipios', index=False)
    df_table_3.to_excel(writer, sheet_name='Estados', index=False)

In [14]:
notification.notify(
        title = 'EXTRAÇÃO CONCLUÍDA!',
        message = 'Os dados foram enviados para um documento excel dentro da pasta.',
        app_name = 'Alert',
        timeout = 10
        )